In [1]:
import getpass
import psycopg2
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [2]:
filepath = '../Resources/Glucose_Datasets/normal_glucose.csv'
medical_df = pd.read_csv(filepath)
medical_df.head(10)

,ID#,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,7,Male,74,1,1,Yes,Private,Rural,70.09,27.4,Never,1
1,8,Female,69,0,0,No,Private,Urban,94.39,22.8,Never,1
2,9,Female,59,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
3,10,Female,78,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1
4,11,Female,81,1,0,Yes,Private,Rural,80.43,29.7,Never,1
5,19,Female,60,0,0,No,Private,Urban,89.22,37.8,Never,1
6,32,Male,42,0,0,Yes,Private,Rural,83.41,25.4,Unknown,1
7,35,Male,48,0,0,No,Govt_job,Urban,84.20,29.7,Never,1
8,36,Female,82,1,1,No,Private,Rural,84.03,26.5,Former,1
9,38,Female,72,1,0,Yes,Private,Rural,74.63,23.1,Former,1


In [3]:
# Drop ID# column
medical_df.drop(columns=['ID#'], inplace=True)
medical_df.head()

,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,Male,74,1,1,Yes,Private,Rural,70.09,27.4,Never,1
1,Female,69,0,0,No,Private,Urban,94.39,22.8,Never,1
2,Female,59,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
3,Female,78,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1
4,Female,81,1,0,Yes,Private,Rural,80.43,29.7,Never,1


In [4]:
medical_df['Stroke'].value_counts()

0    3019
1     112
Name: Stroke, dtype: int64

In [5]:
# Get a list of categorical columns
categorical_columns = medical_df.dtypes[medical_df.dtypes=='object'].index.tolist()
categorical_columns

['Gender', 'Ever_Married', 'Work_Type', 'Residence_Type', 'Smoker']

In [6]:
# Create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit & transform OneHotEncoder using categorical columns
encode_df = pd.DataFrame(enc.fit_transform(medical_df[categorical_columns]))

# Add column names
encode_df.columns = enc.get_feature_names(categorical_columns)
print(encode_df.shape)
encode_df.head(10)

(3131, 15)


C:\Users\remem\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Never_worked,Work_Type_Private,Work_Type_Self-employed,Work_Type_children,Residence_Type_Rural,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
9,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [7]:
# Get encode_df info
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3131 entries, 0 to 3130
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Female            3131 non-null   float64
 1   Gender_Male              3131 non-null   float64
 2   Ever_Married_No          3131 non-null   float64
 3   Ever_Married_Yes         3131 non-null   float64
 4   Work_Type_Govt_job       3131 non-null   float64
 5   Work_Type_Never_worked   3131 non-null   float64
 6   Work_Type_Private        3131 non-null   float64
 7   Work_Type_Self-employed  3131 non-null   float64
 8   Work_Type_children       3131 non-null   float64
 9   Residence_Type_Rural     3131 non-null   float64
 10  Residence_Type_Urban     3131 non-null   float64
 11  Smoker_Current           3131 non-null   float64
 12  Smoker_Former            3131 non-null   float64
 13  Smoker_Never             3131 non-null   float64
 14  Smoker_Unknown          

In [8]:
# Drop redundant columns
encode_df.drop(columns=['Gender_Female', 'Ever_Married_No', 'Residence_Type_Rural'], inplace=True)
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3131 entries, 0 to 3130
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Male              3131 non-null   float64
 1   Ever_Married_Yes         3131 non-null   float64
 2   Work_Type_Govt_job       3131 non-null   float64
 3   Work_Type_Never_worked   3131 non-null   float64
 4   Work_Type_Private        3131 non-null   float64
 5   Work_Type_Self-employed  3131 non-null   float64
 6   Work_Type_children       3131 non-null   float64
 7   Residence_Type_Urban     3131 non-null   float64
 8   Smoker_Current           3131 non-null   float64
 9   Smoker_Former            3131 non-null   float64
 10  Smoker_Never             3131 non-null   float64
 11  Smoker_Unknown           3131 non-null   float64
dtypes: float64(12)
memory usage: 293.7 KB


In [9]:
# Merge encoded df with medical df
medical_df = medical_df.merge(encode_df, left_index=True, right_index=True).drop(categorical_columns, axis=1)
print(medical_df.shape)
medical_df.head(10)

(3131, 18)


,Age,Hypertension,Heart_Disease,Avg_Glucose_Lvl,BMI,Stroke,Gender_Male,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Never_worked,Work_Type_Private,Work_Type_Self-employed,Work_Type_children,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,74,1,1,70.09,27.4,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,69,0,0,94.39,22.8,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,59,0,0,76.15,NaN,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,78,0,0,58.57,24.2,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,81,1,0,80.43,29.7,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,60,0,0,89.22,37.8,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6,42,0,0,83.41,25.4,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,48,0,0,84.20,29.7,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,82,1,1,84.03,26.5,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,72,1,0,74.63,23.1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
medical_df.isnull().sum()

Age                          0
Hypertension                 0
Heart_Disease                0
Avg_Glucose_Lvl              0
BMI                        101
Stroke                       0
Gender_Male                  0
Ever_Married_Yes             0
Work_Type_Govt_job           0
Work_Type_Never_worked       0
Work_Type_Private            0
Work_Type_Self-employed      0
Work_Type_children           0
Residence_Type_Urban         0
Smoker_Current               0
Smoker_Former                0
Smoker_Never                 0
Smoker_Unknown               0
dtype: int64

In [11]:
# Create SimpleImputer instance to replace missing BMI feature values with median BMI
imputer = SimpleImputer(strategy='mean')

# Fit SimpleImputer & transform data
med_transformed = imputer.fit_transform(medical_df)

In [12]:
# Add SimpleImputer outcome to dataframe
med_df_transformed = pd.DataFrame(med_transformed, columns=medical_df.columns)
print(med_df_transformed.shape)
med_df_transformed.head(10)

(3131, 18)


,Age,Hypertension,Heart_Disease,Avg_Glucose_Lvl,BMI,Stroke,Gender_Male,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Never_worked,Work_Type_Private,Work_Type_Self-employed,Work_Type_children,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,74.0,1.0,1.0,70.09,27.400000,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,69.0,0.0,0.0,94.39,22.800000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,59.0,0.0,0.0,76.15,28.392409,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,78.0,0.0,0.0,58.57,24.200000,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,81.0,1.0,0.0,80.43,29.700000,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,60.0,0.0,0.0,89.22,37.800000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6,42.0,0.0,0.0,83.41,25.400000,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,48.0,0.0,0.0,84.20,29.700000,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,82.0,1.0,1.0,84.03,26.500000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,72.0,1.0,0.0,74.63,23.100000,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [13]:
med_df_transformed.isnull().sum()

Age                        0
Hypertension               0
Heart_Disease              0
Avg_Glucose_Lvl            0
BMI                        0
Stroke                     0
Gender_Male                0
Ever_Married_Yes           0
Work_Type_Govt_job         0
Work_Type_Never_worked     0
Work_Type_Private          0
Work_Type_Self-employed    0
Work_Type_children         0
Residence_Type_Urban       0
Smoker_Current             0
Smoker_Former              0
Smoker_Never               0
Smoker_Unknown             0
dtype: int64

In [14]:
# Create feature & target datasets
X = med_df_transformed.drop(columns=['Stroke'])
y = med_df_transformed['Stroke']
print(X.shape)
print(y.shape)

(3131, 17)
(3131,)


In [15]:
# Create empty arrays to store ML results
recalls = []
accuracies = []
precisions = []

# This will take a while. Feel free to adjust range to sample code. Feel free to review exported spreadsheets in github.
for i in range(1000):
    
    # Create training + testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    # Oversample positive stroke cases in training set
    ros = RandomOverSampler()
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    # Scale feature data
    scaler = StandardScaler()
    scaler.fit(X_train_resampled)
    X_train_scaled = scaler.transform(X_train_resampled)
    X_test_scaled = scaler.transform(X_test)
    
    # Create SVC model
    svc_model = SVC(kernel='linear')
    svc_model = svc_model.fit(X_train_scaled, y_train_resampled)
    
    # Evaluate SVC model
    y_pred = svc_model.predict(X_test_scaled)
    
    # Add recall results to list
    recalls.append(recall_score(y_test, y_pred))
    
    # Add precision results to list
    precisions.append(precision_score(y_test, y_pred))
    
    # Add accuracy results to list
    accuracies.append(accuracy_score(y_test, y_pred))
    
    print("Current progress:", i)

Current progress: 0
Current progress: 1
Current progress: 2
Current progress: 3
Current progress: 4
Current progress: 5
Current progress: 6
Current progress: 7
Current progress: 8
Current progress: 9
Current progress: 10
Current progress: 11
Current progress: 12
Current progress: 13
Current progress: 14
Current progress: 15
Current progress: 16
Current progress: 17
Current progress: 18
Current progress: 19
Current progress: 20
Current progress: 21
Current progress: 22
Current progress: 23
Current progress: 24
Current progress: 25
Current progress: 26
Current progress: 27
Current progress: 28
Current progress: 29
Current progress: 30
Current progress: 31
Current progress: 32
Current progress: 33
Current progress: 34
Current progress: 35
Current progress: 36
Current progress: 37
Current progress: 38
Current progress: 39
Current progress: 40
Current progress: 41
Current progress: 42
Current progress: 43
Current progress: 44
Current progress: 45
Current progress: 46
Current progress: 47
Cu

Current progress: 378
Current progress: 379
Current progress: 380
Current progress: 381
Current progress: 382
Current progress: 383
Current progress: 384
Current progress: 385
Current progress: 386
Current progress: 387
Current progress: 388
Current progress: 389
Current progress: 390
Current progress: 391
Current progress: 392
Current progress: 393
Current progress: 394
Current progress: 395
Current progress: 396
Current progress: 397
Current progress: 398
Current progress: 399
Current progress: 400
Current progress: 401
Current progress: 402
Current progress: 403
Current progress: 404
Current progress: 405
Current progress: 406
Current progress: 407
Current progress: 408
Current progress: 409
Current progress: 410
Current progress: 411
Current progress: 412
Current progress: 413
Current progress: 414
Current progress: 415
Current progress: 416
Current progress: 417
Current progress: 418
Current progress: 419
Current progress: 420
Current progress: 421
Current progress: 422
Current pr

Current progress: 751
Current progress: 752
Current progress: 753
Current progress: 754
Current progress: 755
Current progress: 756
Current progress: 757
Current progress: 758
Current progress: 759
Current progress: 760
Current progress: 761
Current progress: 762
Current progress: 763
Current progress: 764
Current progress: 765
Current progress: 766
Current progress: 767
Current progress: 768
Current progress: 769
Current progress: 770
Current progress: 771
Current progress: 772
Current progress: 773
Current progress: 774
Current progress: 775
Current progress: 776
Current progress: 777
Current progress: 778
Current progress: 779
Current progress: 780
Current progress: 781
Current progress: 782
Current progress: 783
Current progress: 784
Current progress: 785
Current progress: 786
Current progress: 787
Current progress: 788
Current progress: 789
Current progress: 790
Current progress: 791
Current progress: 792
Current progress: 793
Current progress: 794
Current progress: 795
Current pr

In [16]:
recalls

[0.8181818181818182,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7727272727272727,
 0.6818181818181818,
 0.7727272727272727,
 0.8181818181818182,
 0.6818181818181818,
 0.6363636363636364,
 0.7727272727272727,
 0.7727272727272727,
 0.6363636363636364,
 0.8181818181818182,
 0.7272727272727273,
 0.6363636363636364,
 0.6818181818181818,
 0.6818181818181818,
 0.6818181818181818,
 0.5454545454545454,
 0.5909090909090909,
 0.8181818181818182,
 0.8181818181818182,
 0.6363636363636364,
 0.7727272727272727,
 0.8636363636363636,
 0.7272727272727273,
 0.7727272727272727,
 0.6363636363636364,
 0.6818181818181818,
 0.6818181818181818,
 0.6363636363636364,
 0.6363636363636364,
 0.6818181818181818,
 0.9090909090909091,
 0.7727272727272727,
 0.6818181818181818,
 0.7727272727272727,
 0.7727272727272727,
 0.6363636363636364,
 0.7272727272727273,
 0.7727272727272727,
 0.7272727272727273,
 0.6363636363636364,
 0.7272727272727273,
 0.6363636363636364,
 0.6818181818181818,
 0.7727272727

In [17]:
recalls_df = pd.DataFrame(recalls, columns=['Recall'])
print(recalls_df.shape)
recalls_df.head(10)

(1000, 1)


,Recall
0,0.818182
1,0.727273
2,0.727273
3,0.727273
4,0.772727
5,0.681818
6,0.772727
7,0.818182
8,0.681818
9,0.636364


In [18]:
accuracies

[0.7783094098883573,
 0.7352472089314195,
 0.733652312599681,
 0.7464114832535885,
 0.7304625199362041,
 0.7320574162679426,
 0.7432216905901117,
 0.7145135566188198,
 0.7830940988835726,
 0.6921850079744817,
 0.7464114832535885,
 0.7416267942583732,
 0.7432216905901117,
 0.7129186602870813,
 0.7432216905901117,
 0.7639553429027113,
 0.7623604465709729,
 0.7639553429027113,
 0.7304625199362041,
 0.7607655502392344,
 0.7942583732057417,
 0.7304625199362041,
 0.7511961722488039,
 0.7527910685805422,
 0.7767145135566188,
 0.7129186602870813,
 0.7432216905901117,
 0.7065390749601276,
 0.6953748006379585,
 0.7464114832535885,
 0.7161084529505582,
 0.7240829346092504,
 0.8006379585326954,
 0.74481658692185,
 0.6985645933014354,
 0.7416267942583732,
 0.7161084529505582,
 0.748006379585327,
 0.7113237639553429,
 0.7320574162679426,
 0.7065390749601276,
 0.7368421052631579,
 0.7304625199362041,
 0.7655502392344498,
 0.7304625199362041,
 0.7161084529505582,
 0.7511961722488039,
 0.69537480063795

In [19]:
accuracies_df = pd.DataFrame(accuracies, columns=['Accuracy'])
print(accuracies_df.shape)
accuracies_df.head(10)

(1000, 1)


,Accuracy
0,0.778309
1,0.735247
2,0.733652
3,0.746411
4,0.730463
5,0.732057
6,0.743222
7,0.714514
8,0.783094
9,0.692185


In [20]:
precisions

[0.11764705882352941,
 0.09090909090909091,
 0.0903954802259887,
 0.09467455621301775,
 0.09392265193370165,
 0.08522727272727272,
 0.09826589595375723,
 0.09326424870466321,
 0.10416666666666667,
 0.07035175879396985,
 0.09941520467836257,
 0.09770114942528736,
 0.08383233532934131,
 0.09278350515463918,
 0.0935672514619883,
 0.09090909090909091,
 0.09554140127388536,
 0.09615384615384616,
 0.0847457627118644,
 0.07894736842105263,
 0.09774436090225563,
 0.09836065573770492,
 0.10588235294117647,
 0.08695652173913043,
 0.1118421052631579,
 0.09693877551020408,
 0.0935672514619883,
 0.08673469387755102,
 0.07106598984771574,
 0.08982035928143713,
 0.08064516129032258,
 0.0782122905027933,
 0.10687022900763359,
 0.08928571428571429,
 0.0966183574879227,
 0.09770114942528736,
 0.08064516129032258,
 0.1,
 0.08808290155440414,
 0.08045977011494253,
 0.08247422680412371,
 0.096045197740113,
 0.0893854748603352,
 0.0915032679738562,
 0.0893854748603352,
 0.07608695652173914,
 0.0914634146341

In [21]:
precisions_df = pd.DataFrame(precisions, columns=['Precision'])
print(precisions_df.shape)
precisions_df.head(10)

(1000, 1)


,Precision
0,0.117647
1,0.090909
2,0.090395
3,0.094675
4,0.093923
5,0.085227
6,0.098266
7,0.093264
8,0.104167
9,0.070352


In [22]:
results_df = accuracies_df.merge(precisions_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 2)


,Accuracy,Precision
0,0.778309,0.117647
1,0.735247,0.090909
2,0.733652,0.090395
3,0.746411,0.094675
4,0.730463,0.093923
5,0.732057,0.085227
6,0.743222,0.098266
7,0.714514,0.093264
8,0.783094,0.104167
9,0.692185,0.070352


In [23]:
results_df = results_df.merge(recalls_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 3)


,Accuracy,Precision,Recall
0,0.778309,0.117647,0.818182
1,0.735247,0.090909,0.727273
2,0.733652,0.090395,0.727273
3,0.746411,0.094675,0.727273
4,0.730463,0.093923,0.772727
5,0.732057,0.085227,0.681818
6,0.743222,0.098266,0.772727
7,0.714514,0.093264,0.818182
8,0.783094,0.104167,0.681818
9,0.692185,0.070352,0.636364


In [24]:
print("Recall mean: " + str(results_df['Recall'].mean()))
print("Recall mode: " + str(results_df['Recall'].mode()))
print("Recall median: " + str(results_df['Recall'].median()))
print("Recall max: " + str(results_df['Recall'].max()))
print("Recall min: " + str(results_df['Recall'].min()))
print("Recall standard deviation: " + str(results_df['Recall'].std()))

Recall mean: 0.7338181818181825
Recall mode: 0    0.727273
dtype: float64
Recall median: 0.7272727272727273
Recall max: 0.9545454545454546
Recall min: 0.4090909090909091
Recall standard deviation: 0.08838595216067578


In [25]:
print("Accuracy mean: " + str(results_df['Accuracy'].mean()))
print("Accuracy mode: " + str(results_df['Accuracy'].mode()))
print("Accuracy median: " + str(results_df['Accuracy'].median()))
print("Accuracy max: " + str(results_df['Accuracy'].max()))
print("Accuracy min: " + str(results_df['Accuracy'].min()))
print("Accuracy standard deviation: " + str(results_df['Accuracy'].std()))

Accuracy mean: 0.73977671451356
Accuracy mode: 0    0.735247
dtype: float64
Accuracy median: 0.7400318979266348
Accuracy max: 0.8006379585326954
Accuracy min: 0.6762360446570973
Accuracy standard deviation: 0.01988893342614954


In [26]:
print("Precision mean: " + str(results_df['Precision'].mean()))
print("Precision mode: " + str(results_df['Precision'].mode()))
print("Precision median: " + str(results_df['Precision'].median()))
print("Precision max: " + str(results_df['Precision'].max()))
print("Precision min: " + str(results_df['Precision'].min()))
print("Precision standard deviation: " + str(results_df['Precision'].std()))

Precision mean: 0.09327492537174698
Precision mode: 0    0.1
dtype: float64
Precision median: 0.09326424870466321
Precision max: 0.13157894736842105
Precision min: 0.05625
Precision standard deviation: 0.010626883136769594


In [27]:
results_df.isnull().sum().sum()

0

In [28]:
len(results_df)

1000

In [29]:
filepath = ('ML_result_files/SVC_NormalGlucose_SimpleImputer(mean)_RandomOverSampler.csv')
results_df.to_csv(filepath, index=False)